
# COMP305 -> 2-median problem on Optimal Placement of 2 Hospitals

## Imports

In [1]:
import time

import heapq
import numpy as np
from collections import defaultdict
from collections import Counter

from random import choice
from random import randint

## Data Read

In [2]:
#with open("tests/test1_new.txt") as f:
 #   test2 = f.read().splitlines()   

#with open("tests/test2_new.txt") as f:
 #   test2 = f.read().splitlines()   
    
#with open("tests/test3_new.txt") as f:
    #test3 = f.read().splitlines() 
    
    
#with open("tests/test1_aycan.txt") as f:
 #   test01 = f.read().splitlines() 
    
    
#with open("tests/test_aycan.txt") as f:
 #   test001 = f.read().splitlines() 

# txt -> Graph

In [3]:
with open("tests/test2_new.txt") as f:
    test2 = f.read().splitlines()   

lines=test2

number_of_vertices = int(lines[0])
number_of_edges = int(lines[1])

vertices = lines[2:2+number_of_vertices]
edges = lines[2+number_of_vertices:]

ids_and_populations = [tuple(map(int, vertices[i].split(" "))) for i in range(len(vertices))]
populations = dict(sorted(dict(ids_and_populations).items())) #redundant sort

mydict = lambda: defaultdict(lambda: defaultdict())
G = mydict()

for i in range(len(edges)):
    source, target, weight = map(int, edges[i].split(" "))
    G[source][target] = weight
    G[target][source] = weight


## Random spawned k-th neighbor Subgraph expansion

In [4]:
def dijkstra_path(G, population_dict, source):
    costs = dict()
    for key in G:
        costs[key] = np.inf
    costs[source] = 0
    #display(source,costs)

    pq = []
    for node in G:
        heapq.heappush(pq, (node, costs[node]))

    while len(pq) != 0:
        current_node, current_node_distance = heapq.heappop(pq)
        for neighbor_node in G[current_node]:
            #print(current_node,costs[source])
            weight = G[current_node][neighbor_node]
            distance = current_node_distance + weight
            if distance < costs[neighbor_node]:
                #if source==neighbor_node:
                    #print('here')
                costs[neighbor_node] = distance
                heapq.heappush(pq, (neighbor_node, distance))
   
    sorted_costs_lst=list(dict(sorted(costs.items())).values())
    sorted_populations_lst = list(dict(sorted(population_dict.items())).values())
    #print(np.array(sorted_costs_lst) ,np.array(sorted_populations_lst))
    return np.array(sorted_costs_lst) * np.array(sorted_populations_lst)
    #return list(dict(sorted(costs.items())).values())
   
   
# V4 because runs in V^4
def V4(G):

    APSP = np.zeros((number_of_vertices,number_of_vertices))
    population_dict = dict(sorted([(k, populations[k]) for k in G.keys()]))
    for vertex in vertices:
        vertex= int(vertex.split()[0])
        APSP[vertex] = [e for e in dijkstra_path(G, population_dict,vertex)]

    global glob

    res = {}

    n = len(APSP)
   
    temp_arr = APSP.copy()

    count=0
    count2=0
    for first in range(n):
        for second in range(first+1,n):
            if first==second:
                continue
            count+=1
            #print(count)
            #print(first,second)
            temp_arr = APSP.copy()
            for row in temp_arr:
                if row[first]<row[second]:
                    row[second]=0
                else:
                    row[first]=0
            #print(temp_arr,count)
            to_be_summed = temp_arr[:,[first,second]]
            summed = sum(sum(to_be_summed))
            res[(first,second)]=summed
    ret=min(res, key=res.get)
    #display(len(res))
    #display(res)
    #print('pick {}th and {}th vertices to place hospitals!'.format(ret[0],ret[1]))
    return ret, res[ret], res

# ML 

In [5]:
class Vertex:

    def __init__(self, id, weight):
        self.key = 0
        self.id = id
        self.visited = False
        self.weight = weight
        self.neighbour_count = 0
        self.neighbour_weight = 0
        self.distance = 0
        self.sum = 0
        self.neighbour_neighbour_count = 0
        self.neighbour_neighbour_weight = 0
        self.neighbour_distance = 0
        self.neighbour_sum = 0
        self.neighbour_list = []
        
    def add_neighbour(self, Vertex, distance):
        self.neighbour_list += [Vertex]
        self.neighbour_count = self.neighbour_count + 1
        self.distance = self.distance + distance
        self.neighbour_weight = self.neighbour_weight + Vertex.weight
        self.sum = self.sum + distance *  Vertex.weight
        self.visited = True
        
    def add_neighbour_neighbour(self, Vertex):
        self.neighbour_neighbour_count +=  Vertex.neighbour_count
        self.neighbour_neighbour_weight += Vertex.neighbour_weight
        self.neighbour_distance +=  Vertex.distance
        self.neighbour_sum += Vertex.sum
        self.visited = True

    def calculate(self, a1, a2, a3):
        self.key = self.weight *  a1 +  self.sum *  a2 +  self.neighbour_sum * a3   
        return self.key

    def get_key(self):
        return self.key
    
    def __str__(self):
        return "Key: " + self.key.__str__() +  " Id: " + self.id.__str__() + \
        " Visited: " + self.visited.__str__() + " Weight: " + self.weight.__str__() +  \
        " Neighbour_count: " + self.neighbour_count.__str__() + " Neighbour_weight: " + self.neighbour_weight.__str__() + \
        " Distance: " + self.distance.__str__() + " Sum: " + self.sum.__str__() + \
        " N_N_count: " + self.neighbour_neighbour_count.__str__() + " N_N_weight: " + self.neighbour_neighbour_weight.__str__() + \
        " N_Distance: " + self.neighbour_distance.__str__() + " N_Sum: " + self.neighbour_sum.__str__() +  "\n"
    
answer = V4(G)    
    
answer=answer[2]

vertex_list = []

for i in range(len(populations)):
    v = Vertex(i, populations[i])
    vertex_list += [v] 

for i in range(len(edges)):
    index1, index2, distance = map(int, edges[i].split(" "))
    v0 = vertex_list[index1]
    v1 = vertex_list[index2]
    v0.add_neighbour(v1, distance)
    v1.add_neighbour(v0, distance)
    

for i in range(len(vertex_list)):
    Vlist = vertex_list[i].neighbour_list
    v0 = vertex_list[i]
    for j in range(len(Vlist)):
        v0.add_neighbour_neighbour(vertex_list[j])    
        
        
        
ret=answer[min(answer.keys())]
ret_val=min(answer.keys())
    
dict2 = {}
arr = []
for (key, value) in answer.items():
    dict2[value] = key
    arr += [value]

In [6]:
with open("tests/test2_new.txt") as f:
    lines = f.read().splitlines()

number_of_vertices = int(lines[0])
number_of_edges = int(lines[1])

vertices = lines[2:2+number_of_vertices]
edges = lines[2+number_of_vertices:]

ids_and_populations = [tuple(map(int, vertices[i].split(" "))) for i in range(len(vertices))]
populations = dict(ids_and_populations)

mydict = lambda: defaultdict(lambda: defaultdict())
G = mydict()

for i in range(len(edges)):
    source, target, weight = map(int, edges[i].split(" "))
    G[source][target] = weight
    G[target][source] = weight

In [7]:
vertex_list = []

for i in range(len(populations)):
    v = Vertex(i, populations[i])
    vertex_list += [v] 

for i in range(len(edges)):
    index1, index2, distance = map(int, edges[i].split(" "))
    v0 = vertex_list[index1]
    v1 = vertex_list[index2]
    v0.add_neighbour(v1, distance)
    v1.add_neighbour(v0, distance)
    

for i in range(len(vertex_list)):
    Vlist = vertex_list[i].neighbour_list
    v0 = vertex_list[i]
    for j in range(len(Vlist)):
        v0.add_neighbour_neighbour(vertex_list[j])
        
        
        
ret=answer[min(answer.keys())]
ret_val=min(answer.keys())
    
dict2 = {}
arr = []
for (key, value) in answer.items():
    dict2[value] = key
    arr += [value]
    
    
        

In [8]:
learning_rate = 1.0e-8
epochs = 10
bias = 1
sum = 0

def Preceptator(Vertex1, Vertex2, a1, a2, a3, output):
    outputP = Vertex1.calculate(a1, a2, a3) + Vertex2.calculate(a1, a2, a3)

    error1 = learning_rate * (output - outputP) * a1 
    error2 = learning_rate * (output - outputP) * a2
    error3 = learning_rate * (output - outputP) * a3

    a1 += error1
    a2 += error2
    a3 += error3
    return abs(output - outputP)

def getClosestValue(k):
    b = int(k)
    lst = list(range(b - 1, b + 2, 1))  # lst = [b-1,b,b+1]
    return lst[min(range(len(lst)), key=lambda i: abs(lst[i] - k))]

def Predict(Vertex1, Vertex2, a1, a2, a3):
    value = Vertex1.calculate(a1, a2, a3) + Vertex2.calculate(a1, a2, a3)
    if (value < 0):
        print("value : ", int(value) - 1)
    elif value == 0:
        print("value : ", value)
    else:
        print("value : ", int(value))
    print("y  =  ", getClosestValue(w[0]), " + ", getClosestValue(w[1]), " * x1 + ", getClosestValue(w[2]), " * x2")
    
def calculate(Vertex, a1, a2, a3, a4, a5, a6, a7, a8, a9):
    return Vertex.weight *  a1 +  Vertex.neighbour_count * a2 + \
    Vertex.neighbour_weight *  a3 +  Vertex.distance * a4 + \
    Vertex.sum *  a5 +  Vertex.neighbour_neighbour_count * a6 + \
    Vertex.neighbour_neighbour_weight *  a7 +  Vertex.neighbour_distance * a8 + \
    Vertex.neighbour_sum * a9
    
def sigmoid(X, w, w0):
    return (1 / (1 + np.exp(-(np.matmul(X, w) + w0))))


def gradient_W(X, y_truth, y_predicted):
    return (np.asarray(
        [-np.sum(np.repeat((y_truth[:, c] - y_predicted[:, c])[:, None], X.shape[1], axis=1) * X, axis=0) for c in
         range(K)]).transpose()) / 3

def gradient_w0(Y_truth, Y_predicted):
    return (-np.sum(Y_truth - Y_predicted, axis=0))

In [ ]:
# training sets
#for i in range(epochs):
a1 = np.random.rand(1) * 100 
a2 = np.random.rand(1) * 10 
a3 = np.random.rand(1) * 1 
count = 0

for i in range(len(vertex_list)):
    for j in range(int(len(vertex_list))):   
        if i>j:       
            v0 = vertex_list[i]
            v1 = vertex_list[j]
            value = arr[count]
            count += 1
            for j in range(epochs):
                val = Preceptator(v0, v1, a1, a2, a3, value)
                if j == epochs-1:
                    sum += val
            
print(sum/(count)/max(arr))


#Normalizing the variables
coef_sum = a1 + a2 + a3
a1 = a1 / coef_sum
a2 = a2 / coef_sum
a3 = a3 / coef_sum

In [ ]:
#with open("tests/test3_new.txt") as f:
    #lines = f.read().splitlines()
    
with open("tests/test3_new.txt") as f:
    lines = f.read().splitlines()


number_of_vertices = int(lines[0])
number_of_edges = int(lines[1])

vertices = lines[2:2+number_of_vertices]
edges = lines[2+number_of_vertices:]

ids_and_populations = [tuple(map(int, vertices[i].split(" "))) for i in range(len(vertices))]
populations = dict(ids_and_populations)

mydict = lambda: defaultdict(lambda: defaultdict())
G = mydict()

for i in range(len(edges)):
    source, target, weight = map(int, edges[i].split(" "))
    G[source][target] = weight
    G[target][source] = weight
    



In [ ]:
start = time.time()



vertex_list = []
for i in range(len(populations)):
    v = Vertex(i, populations[i])
    vertex_list += [v] 

for i in range(len(edges)):
    index1, index2, distance = map(int, edges[i].split(" "))
    v0 = vertex_list[index1]
    v1 = vertex_list[index2]
    v0.add_neighbour(v1, distance)
    v1.add_neighbour(v0, distance)
    

for i in range(len(vertex_list)):
    Vlist = vertex_list[i].neighbour_list
    v0 = vertex_list[i]
    for j in range(len(Vlist)):
        v0.add_neighbour_neighbour(vertex_list[j])

In [ ]:
a1 = 92.72
a2 = 9.87
a3 = 0.89

for i in range(len(vertex_list)):    
    v0 = vertex_list[i]
    v0.calculate(a1, a2, a3)
    
K_arr = []
for i in range(len(vertex_list)):        
    K_arr += [vertex_list[i].get_key()]

min_val = min(K_arr) 
K_arr.sort()
index1 = 0
index2 = 0

for i in range(len(vertex_list)):
    v = vertex_list[i]
    if K_arr[0] == v.get_key():
        index1 = i
    if K_arr[1] == v.get_key():
        index2 = i
    

diff = time.time()-start



print('time took: '+str(diff))
print('first node:{}, second node:{} ; associated cost:{}'.format(index1,index2,min_val))

In [ ]:
def select_neighbors(G, sub_graph, current_node, k):
    if k == 0:
        return sub_graph

    for j in G[current_node].items():
        sub_graph[current_node][j[0]] = j[1]
        sub_graph[j[0]][current_node] = j[1]

        sub_graph = select_neighbors(G, sub_graph, j[0], k - 1)
    return sub_graph


def merge_graph(dict1, dict2):
    for key, value in dict2.items():
        for subkey, subvalue in value.items():
            dict1[key][subkey] = subvalue


def dijkstra_q_impl(G, populations, source): 
    costs = dict()
    for key in G:
        costs[key] = np.inf
    costs[source] = 0

    pq = []
    for node in G:
         pq.append((node, costs[node]))

    while len(pq) != 0:
        current_node, current_node_distance = pq.pop(0)
        for neighbor_node in G[current_node]:
            weight = G[current_node][neighbor_node]
            distance = current_node_distance + weight
            if distance < costs[neighbor_node]:
                costs[neighbor_node] = distance
                pq.append((neighbor_node, distance))
    
    #return (costs.values(),population_dict[])
    sorted_costs_lst=list(dict(sorted(costs.items())).values())
    populations_values_lst = list(dict(sorted(populations.items())).values())
    return np.sum(np.array(sorted_costs_lst) * np.array(populations_values_lst))

def random_start(G):
    res = [choice(list(G.keys())), choice(list(G.keys()))]
    if res[0] == res [1]:
        return random_start(G)
    print(f"Random start: {res}")
    return res
    #return [929940, 301820]


#//2 * O((V+E)*logV) = O(E*logV) // 
def allocation_cost(G, population_dict, i,j):
    return [dijkstra_q_impl(G,population_dict, i),dijkstra_q_impl(G,population_dict, j)]


# V times Dijkstra
def sub_graph_apsp(G, dijkstra_func):
    population_dict = dict(sorted([(k, populations[k]) for k in G.keys()]))
    selected_vertex = choice(list(G.keys()))
    selected_cost = dijkstra_func(G,population_dict, selected_vertex)
    
    for node in G.keys():
        if node is not selected_vertex:
            this_cost = dijkstra_func(G, population_dict, node) 
            if this_cost < selected_cost:
                selected_cost = this_cost
                selected_vertex = node    
    return selected_vertex, selected_cost


def algorithm_sub_graph_apsp(G, starting_node, k, hop_list, dijkstra_func):
    sub_graph = lambda: defaultdict(lambda: defaultdict())
    sub_graph = sub_graph()
    sub_graph = select_neighbors(G, sub_graph, current_node=starting_node, k=k)
    next_node, cost = sub_graph_apsp(sub_graph, dijkstra_func)
    #print(next_node)
    
    if len(hop_list) > 0 and next_node == hop_list[-1][0]:
        return next_node, cost
    
    hop_list.append((next_node, cost))
    return algorithm_sub_graph_apsp(G, next_node, k, hop_list, dijkstra_func)



In [ ]:
# 2*O(V)*O(E*logV) = O(E*V*logV) #
def Greedy_Heuristic_Add_Drop(G, dijkstra_func):
    population_dict = dict(sorted([(k, populations[k]) for k in G.keys()]))
    #population_dict = [populations[i] for i in G.keys()]
    selected_vertices = random_start(G) 
    selected_costs = allocation_cost(G,population_dict, selected_vertices[0],selected_vertices[1])
    
    for not_selected in G.keys():
        if not_selected not in selected_vertices:
            bigger = max(selected_costs)
            this_cost = dijkstra_func(G,population_dict, not_selected) 
            if this_cost < bigger:
                bigger_index = selected_costs.index(bigger)
                selected_costs[bigger_index] = this_cost
                selected_vertices[bigger_index] = not_selected
    return(selected_vertices,selected_costs)


def Greedy_Heuristic_Subgraph_Expansion(G, k, dijkstra_func, bootstrap_cnt=10):
    nodes = []
    costs = []
    
    for i in range(bootstrap_cnt):
        #print("iter")
        node, cost = algorithm_sub_graph_apsp(G, choice(list(G.keys())), k, [], dijkstra_func=dijkstra_func)
        nodes.append(node)
        costs.append(cost)
        
    counter = Counter(nodes)
    most_commons = counter.most_common(2)
    target_nodes = (most_commons[0][0], most_commons[1][0])
    
    sub_graph1 = lambda: defaultdict(lambda: defaultdict())
    sub_graph1 = sub_graph1()
    sub_graph1 = select_neighbors(G, sub_graph1, target_nodes[0], k=k)
    
    sub_graph2 = lambda: defaultdict(lambda: defaultdict())
    sub_graph2 = sub_graph2()
    sub_graph2 = select_neighbors(G, sub_graph2, target_nodes[1], k=k)

    merge_graph(sub_graph1, sub_graph2)

    points, costs = Greedy_Heuristic_Add_Drop(sub_graph1, dijkstra_func)

    if np.inf in costs:
        print("INF")
        sub_graph1 = lambda: defaultdict(lambda: defaultdict())
        sub_graph1 = sub_graph1()
        sub_graph1 = select_neighbors(G, sub_graph1, current_node=points[0], k=k+1)
        
        sub_graph2 = lambda: defaultdict(lambda: defaultdict())
        sub_graph2 = sub_graph2()
        sub_graph2 = select_neighbors(G, sub_graph2, current_node=points[1], k=k+1)
        
        merge_graph(sub_graph1, sub_graph2)
        
        

        points, costs = Greedy_Heuristic_Add_Drop(sub_graph1, dijkstra_func)


            
            
        if np.inf not in costs:
            return points, costs
        else:
            print("Graphs are disconnected. Total cost is inf")
            return points, costs
    return points, costs


In [ ]:

start = time.time()

res = Greedy_Heuristic_Subgraph_Expansion(G, 5, bootstrap_cnt=10, dijkstra_func=dijkstra_q_impl) #q for direct Queue based PQ impl (py's pop(0))

diff = time.time()-start

print('\npick cities #'+  str(res[0]) +' with costs '+ str(res[1]))
print('\ntotal time using our Queue-based PQ: '+ str(diff)+ ' sec')